# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [23]:
!pip install langchain_chroma

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
Using cached langchain_chroma-0.1.4-py3-none-any.whl (10 kB)
Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl (413 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.2
    Uninstalling protobuf-5.28.2:
      Successfully uninstalled protobuf-5.28.2


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.66.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.


In [1]:
import os
import dotenv
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

dotenv.load_dotenv()

True

In [2]:
loader = TextLoader('acme_tech_rag.txt', encoding='utf-8')
doc = loader.load()
doc

[Document(metadata={'source': 'acme_tech_rag.txt'}, page_content='\ufeffQ: Who founded Acme Technologies Ltd?\nA: Acme Technologies Ltd was founded by a group of retired military officers and engineers.\nQ: Who I am?\nA: I am a chatbot designed for ACME Technologies Ltd to answer HR-related questions about their history, departments, job opportunities, policies, allowances, projects and more\nQ: What was Acme Technologies Ltd\'s initial focus as a company?\nA: The company initially started as a modest real estate developer.\nQ: How did the company expand into the construction sector?\nA: Acme Technologies Ltd expanded into constructing industrial and commercial complexes under the name Acme Constructions.\nQ: What business ethics does Acme Technologies Ltd follow to ensure its success?\nA: The company follows basic business ethics such as timely delivery, maintaining the quality of works, and focusing on stunning architecture.\nQ: How did Acme Technologies Ltd gain prominence in the in

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=250
)
texts = text_splitter.split_documents(doc)
texts

[Document(metadata={'source': 'acme_tech_rag.txt'}, page_content="\ufeffQ: Who founded Acme Technologies Ltd?\nA: Acme Technologies Ltd was founded by a group of retired military officers and engineers.\nQ: Who I am?\nA: I am a chatbot designed for ACME Technologies Ltd to answer HR-related questions about their history, departments, job opportunities, policies, allowances, projects and more\nQ: What was Acme Technologies Ltd's initial focus as a company?\nA: The company initially started as a modest real estate developer.\nQ: How did the company expand into the construction sector?\nA: Acme Technologies Ltd expanded into constructing industrial and commercial complexes under the name Acme Constructions.\nQ: What business ethics does Acme Technologies Ltd follow to ensure its success?\nA: The company follows basic business ethics such as timely delivery, maintaining the quality of works, and focusing on stunning architecture.\nQ: How did Acme Technologies Ltd gain prominence in the ind

In [11]:
len(texts)

90

In [12]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
import chromadb

In [13]:
client = chromadb.PersistentClient(path='vectordb')
collection = client.get_or_create_collection(name='acme_tech')


In [14]:
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\shahabuddin akhon hr\AppData\Local\Temp\ipykernel_16796\2213097150.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\shahabuddin akhon hr\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


C:\Users\shahabuddin akhon hr\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
# Embed the documents' content
embeddings = embedding_model.embed_documents([doc.page_content for doc in texts])


In [16]:
for i, doc in enumerate(texts):
    print(i)
    collection.add(
        ids=[str(i)],  # Unique ID for each document
        documents=[doc.page_content],
        embeddings=[embeddings[i]],
        metadatas=[doc.metadata]  # Optionally, include metadata
    )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


In [17]:
retriever = collection.query(
    query_texts=["Tell me something about acme technology"],  # Your query
    n_results=2  # Number of results you want
)
print(retriever)


{'ids': [['1', '0']], 'distances': [[0.7690942016509594, 0.8360207396156266]], 'metadatas': [[{'source': 'acme_tech_rag.txt'}, {'source': 'acme_tech_rag.txt'}]], 'embeddings': None, 'documents': [["A: Through its commitment to business ethics, Acme Technologies Ltd gained prominence and came into the limelight.\nQ: Why did Acme Technologies Ltd create a new company, Acme Services?\nA: To meet its own needs and the needs of its clients in the service sector, Acme Technologies Ltd created a separate company called Acme Services.\nQ: What new business sector has Acme Technologies Ltd recently ventured into?\nA: The company has recently ventured into the field of information technology.\nQ: What is the name of Acme Technologies Ltd's IT sister concern?\nA: The IT division of the company is called Acme AI.\nQ: What is Acme AI primarily focused on right now?\nA: Acme AI is primarily focused on annotation services at the moment.\nQ: What are Acme AI’s future plans in the IT field?\nA: In the 

Make a retriever

In [22]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv('GROQ_API_KEY'),
    model="llama-3.1-8b-instant",
)

In [55]:
query_text = 'Tell me about who works in the Sales & Marketing department in details'

retriever = collection.query(
    query_texts=query_text,  # Your query
    n_results=2  # Number of results you want
).get('documents')
retriever

[['Q: Tell me about who works in the Sales & Marketing department.\nA:\n   . Director: Shadab Iqbal Siddique, Phone No: +8801920-667788, Reports to: Maj Gen Syed Fatemy Ahmed Roomy, Retd\n   . Assistant Manager: Abu Salim, Phone No: +8801704-118050, Reports to: Shadab Iqbal Siddique\n   . Sr. Executive: Kishore Kumar Sarker, Phone No: +8801704-118052, Reports to: Shadab Iqbal Siddique\n   . Marketing Officer: Md. Minhazul Muntasir Kafi, Phone No: +8801704-118053, Reports to: Shadab Iqbal Siddique\nQ: Tell me about who works in the Acme Services.\nA:\n   . AGM: Md. Akharuzzaman, Phone No: +8801704-118060, Reports to: Maj Gen Syed Fatemy Ahmed Roomy, Retd\n   . Shutter In-Charge: Md. Zillur Rahman, Phone No: +881 7 041 18063, Reports to: Md. Akharuzzaman\n   . Warehouse Officer: Engineer-1 & Ot-2\nQ: Tell me about who works in the Budget department.\nA:\n   . Sr. Executive: Md. Khurshed Alam, Phone No: +8801704-118084, Reports to: Maj Gen Syed Fatemy Ahmed Roomy, Retd\n   . Budget Office

In [56]:
from langchain_core.prompts import PromptTemplate

# Define the template that takes in the user question and the retrieved document
template = """
You are an expert in HR topics. The user has a question, and you will answer it based on the relevant information provided from the HR manual.

User question: {user_question}
Relevant information: {answer}

Provide a specific and accurate response based on the relevant information above.(NO PREAMBLE)
"""
# Create the PromptTemplate
prompt_template = PromptTemplate.from_template(
    template
)


In [59]:
chain = prompt_template | llm
res = chain.invoke(input={'user_question': query_text, 'answer': [retriever[0][0], retriever[0][1]]})
print(res.content)

The Sales & Marketing department consists of the following employees:

1. Director: Shadab Iqbal Siddique, Phone No: +8801920-667788, Reports to: Maj Gen Syed Fatemy Ahmed Roomy, Retd
2. Assistant Manager: Abu Salim, Phone No: +8801704-118050, Reports to: Shadab Iqbal Siddique
3. Sr. Executive: Kishore Kumar Sarker, Phone No: +8801704-118052, Reports to: Shadab Iqbal Siddique
4. Marketing Officer: Md. Minhazul Muntasir Kafi, Phone No: +8801704-118053, Reports to: Shadab Iqbal Siddique
